### Etapas iniciais: importação de pacotes, definição de funções, carregamento de dados

In [1]:
import pandas as pd

In [2]:
def values_missing(df, only_missing=True):
    '''Calcula e retorna o percentual e o número de registros nulos no dataframe df'''
    missing = pd.concat({'percent_missing': 100* df.isnull().sum() / len(df),
                         'rows_missing': df.isnull().sum()}, axis=1)
    if only_missing:
        missing = missing[missing.percent_missing > 0]
    return missing.sort_values(by='percent_missing', ascending=False)


In [3]:
# Carregamento dos datasets
vivino_wines = pd.read_csv('datasets/vivino_wines.csv')
vivino_vintages = pd.read_csv('datasets/vivino_vintages.csv')

## Dataset: Vivino Wines

In [4]:
# Dataset com os dados de vinhos
vivino_wines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343176 entries, 0 to 343175
Data columns (total 18 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               343176 non-null  int64  
 1   full_name        343176 non-null  object 
 2   name             343175 non-null  object 
 3   type_id          343176 non-null  int64  
 4   type             343176 non-null  object 
 5   wine_region      343176 non-null  object 
 6   winery           343176 non-null  object 
 7   winery_region    343176 non-null  object 
 8   country          343176 non-null  object 
 9   ratings_count    343176 non-null  int64  
 10  ratings_average  343176 non-null  float64
 11  alcohol          343176 non-null  float64
 12  acidity          343176 non-null  float64
 13  fizziness        20842 non-null   float64
 14  intensity        343176 non-null  float64
 15  sweetness        322335 non-null  float64
 16  tannin           210600 non-null  floa

### Filtragem
- O dataset originalmente continha 343.176 registros, os quais cobriam diversos tipos/categorias, como vinhos brancos, espumantes e tintos. Como o trabalho restringe-se a vinhos tintos, nessa etapa foram filtrados apenas os vinhos dessa categoria resultando em 210.432 registros.

In [5]:
vivino_wines = vivino_wines[vivino_wines['type_id'] == 1]

In [6]:
vivino_wines.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 210432 entries, 1 to 343175
Data columns (total 18 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               210432 non-null  int64  
 1   full_name        210432 non-null  object 
 2   name             210431 non-null  object 
 3   type_id          210432 non-null  int64  
 4   type             210432 non-null  object 
 5   wine_region      210432 non-null  object 
 6   winery           210432 non-null  object 
 7   winery_region    210432 non-null  object 
 8   country          210432 non-null  object 
 9   ratings_count    210432 non-null  int64  
 10  ratings_average  210432 non-null  float64
 11  alcohol          210432 non-null  float64
 12  acidity          210432 non-null  float64
 13  fizziness        9 non-null       float64
 14  intensity        210432 non-null  float64
 15  sweetness        210424 non-null  float64
 16  tannin           210390 non-null  floa

### Tratamento de dados duplicados
- Foram encontrados e removidos 1.666 registros inteiramente duplicados.
- Após isso verificou-se a existencia de 1.370 registros duplicados pelo campo **full_name**. Ao analisar pontualmente os registros duplicados, acessando também as páginas correspondentes aos vinho no site do Vivino, pode-se perceber que que os vinhos estão duplicados no site do Vivino, sendo que em alguns casos ao acessar uma das urls é feito o redirecionamento da página para a outra, indicando possivelmente tratar-se de um registro ser o da atual e o outro um antigo. Desta forma optou-se por manter apenas o registro com maior número de análises feitas pelos usuários (**ratings_count**), por entender que o registro com maior número de avaliações é o mais atual. Os 1.370 foram removidos com a aplicação desse entendimento.

In [7]:
# Remoção de registros inteiramente duplicados
print('vivino_wines: Foram encontrados {} registros inteiramente duplicados'.format(vivino_wines.duplicated().sum()))

vivino_wines = vivino_wines.drop_duplicates()
print('Total de registros após a remoção dos registro duplicados: {}'.format(vivino_wines.shape[0]))

vivino_wines: Foram encontrados 1666 registros inteiramente duplicados
Total de registros após a remoção dos registro duplicados: 208766


In [8]:
# Registros duplicados pelo **full_name**
vivino_wines[ vivino_wines.duplicated(subset=['full_name'], keep=False) ].sort_values(by='full_name').tail(50)

,id,full_name,name,type_id,type,wine_region,winery,winery_region,country,ratings_count,ratings_average,alcohol,acidity,fizziness,intensity,sweetness,tannin,url
2445,6876,Windstream - Windbreak Sarmento Vineyard Pinot...,Sarmento Vineyard Pinot Noir,1,Red Wine,Santa Lucia Highlands,Windstream - Windbreak,Central Coast,United States,11,3.6,14.5,3.500000,NaN,3.000000,1.000000,2.000000,https://www.vivino.com/GB/en/windstream-windbr...
172429,2318402,Windstream - Windbreak Sarmento Vineyard Pinot...,Sarmento Vineyard Pinot Noir,1,Red Wine,Santa Lucia Highlands,Windstream - Windbreak,Central Coast,United States,71,3.9,0.0,3.797949,NaN,3.079829,1.000000,2.672992,https://www.vivino.com/GB/en/windstream-windbr...
59214,1407923,Windstream - Windbreak Sarmento Vineyard Pinot...,Sarmento Vineyard Pinot Noir,1,Red Wine,Santa Lucia Highlands,Windstream - Windbreak,Central Coast,United States,71,3.3,0.0,3.711818,NaN,3.318283,1.667273,2.134646,https://www.vivino.com/GB/en/windstream-windbr...
69060,1487150,Wine Spots Pinot Noir,Pinot Noir,1,Red Wine,Los Carneros,Wine Spots,Napa Valley,United States,406,3.9,0.0,3.543049,NaN,3.099103,1.696861,2.377489,https://www.vivino.com/GB/en/wine-spots-pinot-...
327087,6421614,Wine Spots Pinot Noir,Pinot Noir,1,Red Wine,Sonoma Coast,Wine Spots,Napa Valley,United States,60,4.2,0.0,3.310513,NaN,3.250257,1.594530,2.010085,https://www.vivino.com/GB/en/wine-spots-pinot-...
118387,1841270,Wine X Sam - Sam Plunkett The Victorian Shiraz,The Victorian Shiraz,1,Red Wine,Glenrowan,Wine X Sam - Sam Plunkett,Victoria,Australia,510,3.7,0.0,3.077784,NaN,4.761821,2.171410,3.034886,https://www.vivino.com/GB/en/wine-x-sam-sam-pl...
98728,1670079,Wine X Sam - Sam Plunkett The Victorian Shiraz,The Victorian Shiraz,1,Red Wine,Strathbogie Ranges,Wine X Sam - Sam Plunkett,Victoria,Australia,51,3.6,0.0,2.822056,NaN,4.687850,2.254299,2.716449,https://www.vivino.com/GB/en/wine-x-sam-sam-pl...
246457,4162754,Winemaker's Choice Cabernet Sauvignon,Cabernet Sauvignon,1,Red Wine,Central Valley,Winemaker's Choice,Côtes-du-Rhône,France,92,2.8,0.0,3.000000,NaN,4.500000,1.000000,3.500000,https://www.vivino.com/GB/en/winemaker-s-choic...
264453,4625465,Winemaker's Choice Cabernet Sauvignon,Cabernet Sauvignon,1,Red Wine,South Eastern Australia,Winemaker's Choice,Côtes-du-Rhône,France,44,3.3,0.0,3.303834,NaN,4.906833,2.035667,3.384833,https://www.vivino.com/GB/en/winemaker-s-choic...
242240,3993163,Winemaker's Choice Malbec,Malbec,1,Red Wine,Mendoza,Winemaker's Choice,Côtes-du-Rhône,France,182,3.4,0.0,2.500000,NaN,3.985766,1.793358,1.897518,https://www.vivino.com/GB/en/winemaker-s-choic...


In [9]:
# Imprimindo para verificação as URLs de alguns dos vinhos duplicados pelo atributo full_name
url_ids = [237297, 247021, 69060, 327087, 224677, 245179, 172785, 54512, 223193, 21787]

with pd.option_context('display.max_colwidth', 150):
    for url in url_ids:
        print('{}\n{}'.format(vivino_wines.loc[url, ['full_name']], vivino_wines.loc[url, ['url']]))

full_name    18401 Cellars Proprietary Red
Name: 237297, dtype: object
url    https://www.vivino.com/GB/en/onehundredeightyfour-zero-one-cellars-proprietary-red-us-walla-walla-valley/w/3803882
Name: 237297, dtype: object
full_name    18401 Cellars Proprietary Red
Name: 247021, dtype: object
url    https://www.vivino.com/GB/en/onehundredeightyfour-zero-one-cellars-proprietary-red/w/4187752
Name: 247021, dtype: object
full_name    Wine Spots Pinot Noir
Name: 69060, dtype: object
url    https://www.vivino.com/GB/en/wine-spots-pinot-noir-los-carneros/w/1487150
Name: 69060, dtype: object
full_name    Wine Spots Pinot Noir
Name: 327087, dtype: object
url    https://www.vivino.com/GB/en/wine-spots-pinot-noir/w/6421614
Name: 327087, dtype: object
full_name    Winking Owl Shiraz
Name: 224677, dtype: object
url    https://www.vivino.com/GB/en/winking-owl-shiraz/w/1152892
Name: 224677, dtype: object
full_name    Winking Owl Shiraz
Name: 245179, dtype: object
url    https://www.vivino.com/GB/en/wi

In [10]:
vivino_wines[vivino_wines.duplicated(subset=['full_name'],keep=False)]\
            .sort_values(by=['full_name', 'ratings_count'],ascending=[True, False])

,id,full_name,name,type_id,type,wine_region,winery,winery_region,country,ratings_count,ratings_average,alcohol,acidity,fizziness,intensity,sweetness,tannin,url
247021,4187752,18401 Cellars Proprietary Red,Proprietary Red,1,Red Wine,Walla Walla Valley,18401 Cellars,Oregon,United States,68,4.3,0.0,3.677327,NaN,3.655050,1.458713,3.206634,https://www.vivino.com/GB/en/onehundredeightyf...
237297,3803882,18401 Cellars Proprietary Red,Proprietary Red,1,Red Wine,Walla Walla Valley,18401 Cellars,Oregon,United States,42,4.2,0.0,4.072941,NaN,3.839411,1.578235,3.470000,https://www.vivino.com/GB/en/onehundredeightyf...
260788,1181992,3 Stones Pinot Noir,Pinot Noir,1,Red Wine,Central Valley,3 Stones,Marlborough,New Zealand,222,3.4,14.0,3.215273,NaN,3.026637,1.542000,1.392818,https://www.vivino.com/GB/en/three-stones-pino...
124566,83508,3 Stones Pinot Noir,Pinot Noir,1,Red Wine,Marlborough,3 Stones,Marlborough,New Zealand,32,3.7,13.0,3.262821,NaN,2.861538,1.314744,1.860128,https://www.vivino.com/GB/en/three-stones-pino...
78982,1562698,8th Generation Pinot Noir,Pinot Noir,1,Red Wine,Okanagan Valley,8th Generation,Okanagan Valley,Canada,118,3.8,13.6,3.980962,NaN,2.439251,1.000000,2.419626,https://www.vivino.com/GB/en/eight-th-generati...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243568,4039842,Ziata Pinot Noir,Pinot Noir,1,Red Wine,Russian River Valley,Ziata,Napa Valley,United States,46,4.0,14.1,3.470476,NaN,2.832976,1.139643,2.003869,https://www.vivino.com/GB/en/ziata-pinot-noir-...
300487,5711914,Zilzie Wines The Wine Press Series Pinot Noir,The Wine Press Series Pinot Noir,1,Red Wine,Murray Darling,Zilzie Wines,Murray Darling,Australia,39,3.1,0.0,3.195309,NaN,2.864321,1.215309,1.600494,https://www.vivino.com/GB/en/zilzie-wines-the-...
145775,2046503,Zilzie Wines The Wine Press Series Pinot Noir,The Wine Press Series Pinot Noir,1,Red Wine,Sonoma Coast,Zilzie Wines,Murray Darling,Australia,29,3.3,0.0,3.500000,NaN,3.000000,1.000000,2.000000,https://www.vivino.com/GB/en/zilzie-wines-the-...
157659,2203665,l'Arca Pinot Noir,Pinot Noir,1,Red Wine,delle Venezie,l'Arca,Maremma Toscana,Italy,175,3.7,0.0,3.521200,NaN,2.548720,1.137760,1.922720,https://www.vivino.com/GB/en/l-arca-pinot-noir...


In [11]:
vivino_wines = vivino_wines.sort_values(by=['full_name', 'ratings_count'], ascending=[True, False])\
            .drop_duplicates(subset=['full_name'], keep='first')

### Tratamento de dados omissos
- Foram encontrados valores omissos em 4 colunas e dados os seguintes tratamentos:
 - **Fizziness**: a coluna em questão indica o nível de presença de gás (efervescência) do vinho espumante em uma escala de 1 (delicado) a 5 (com gás). Tal medida é própria para espumantes e isso explica o porquê da ausência de 99.99% do preenchimento do campo após a filtragem do dataset para vinhos tintos. A coluna foi removida do dataset.
 - **Tannin**: Os 41 registros omissos (0,02%) foram preenchidos com a média.
 - **Sweetness**: Os 8 registros omissos (0.004%) foram preenchidos com a média.
 - **Name**: Apenas 1 registro omisso. Ao analisar a imagem do rótulo do vinho no site do Vivino percebeu-se que o nome do vinho é 'NA' e estava sendo interpretado como registro nulo. O registro foi corrigido para o nome correto.

In [12]:
values_missing(vivino_wines)

,percent_missing,rows_missing
fizziness,99.995660,207387
tannin,0.019769,41
sweetness,0.003857,8
name,0.000482,1


In [13]:
# Remoção da coluna fizziness - própria de espumantes
vivino_wines = vivino_wines.drop('fizziness', axis=1)

In [14]:
# Tannin: registros omissos preenchidos com a média
vivino_wines['tannin'] = vivino_wines['tannin'].fillna(vivino_wines['tannin'].mean())
# Sweetness: registros omissos preenchidos com a média
vivino_wines['sweetness'] = vivino_wines['sweetness'].fillna(vivino_wines['sweetness'].mean())

In [15]:
# Name
vivino_wines[vivino_wines['name'].isnull()]

,id,full_name,name,type_id,type,wine_region,winery,winery_region,country,ratings_count,ratings_average,alcohol,acidity,intensity,sweetness,tannin,url
220170,3155521,Monte da Ravasqueira NA,NaN,1,Red Wine,Alentejo,Monte da Ravasqueira,Alentejo,Portugal,113,3.7,13.5,2.9057,4.0772,1.7391,2.6465,https://www.vivino.com/GB/en/monte-da-ravasque...


In [16]:
# URL do vinho com nome ausente
with pd.option_context('display.max_colwidth', 150):
    print(vivino_wines[vivino_wines['name'].isnull()].url)

220170    https://www.vivino.com/GB/en/monte-da-ravasqueira-na/w/3155521
Name: url, dtype: object


In [17]:
# Corrigindo nome "ausente"
vivino_wines.loc[220170, 'name'] = 'Monte da Ravasqueira NA'
vivino_wines.loc[[220170]]

,id,full_name,name,type_id,type,wine_region,winery,winery_region,country,ratings_count,ratings_average,alcohol,acidity,intensity,sweetness,tannin,url
220170,3155521,Monte da Ravasqueira NA,Monte da Ravasqueira NA,1,Red Wine,Alentejo,Monte da Ravasqueira,Alentejo,Portugal,113,3.7,13.5,2.9057,4.0772,1.7391,2.6465,https://www.vivino.com/GB/en/monte-da-ravasque...


In [18]:
values_missing(vivino_wines)

,percent_missing,rows_missing


### Remoção de colunas
- As colunas **type_id**, **type** e **url** foram removidas do dataset. As colunas de tipo foram usadas para filtrar o dataset para somente vinhos tintos, portanto pode ser removida pois os valores das colunas possuem sempreo mesmo valor. A coluna url foi gravada para facilitar identificar possíveis erros oriundos da coleta e não terá mais uso de agora em diante.

In [19]:
vivino_wines = vivino_wines.drop(['type_id', 'type', 'url'], axis=1)

In [20]:
## Dos 210.432 registros de vinhos tintos inicialmente (após a filtragem), após a tratamento dos registros duplicados 
## e omissos têm-se 207.396 registros.
vivino_wines.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 207396 entries, 40773 to 172196
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               207396 non-null  int64  
 1   full_name        207396 non-null  object 
 2   name             207396 non-null  object 
 3   wine_region      207396 non-null  object 
 4   winery           207396 non-null  object 
 5   winery_region    207396 non-null  object 
 6   country          207396 non-null  object 
 7   ratings_count    207396 non-null  int64  
 8   ratings_average  207396 non-null  float64
 9   alcohol          207396 non-null  float64
 10  acidity          207396 non-null  float64
 11  intensity        207396 non-null  float64
 12  sweetness        207396 non-null  float64
 13  tannin           207396 non-null  float64
dtypes: float64(6), int64(2), object(6)
memory usage: 33.7+ MB


## Dataset: Vivino Vintages

In [21]:
# Dataset com os dados de safras/ano de engarrafamento
vivino_vintages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1290886 entries, 0 to 1290885
Data columns (total 5 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   year                     1290886 non-null  int64  
 1   id                       1290886 non-null  int64  
 2   wine_id                  1290886 non-null  int64  
 3   vintage_ratings_count    1290886 non-null  int64  
 4   vintage_ratings_average  1290886 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 49.2 MB


### Filtragem
- nenhuma filtragem foi necessária, pois após a unificação de todos os datasets do Vivino, a filtragem feita no dataset Vivino Wines garantirá que apenas vinhos tintos estejam no dataset final.

### Tratamento de dados duplicados
- Foram encontrados e removidos 9418 registros inteiramente duplicados. Reduzind os 1.290.886 registros iniciais para 1.281.468 registros.

In [22]:
# Remoção de registros inteiramente duplicados
print('vivino_vintages: Foram encontrados {} registros inteiramente duplicados'.format(vivino_vintages.duplicated().sum()))

vivino_vintages = vivino_vintages.drop_duplicates()
print('Total de registros após a remoção dos registro duplicados: {}'.format(vivino_vintages.shape[0]))

vivino_vintages: Foram encontrados 9418 registros inteiramente duplicados
Total de registros após a remoção dos registro duplicados: 1281468


In [23]:
print('vivino_vintages: Foram encontrados {} registros duplicados pelo ID'\
      .format(vivino_vintages.duplicated(subset=['id']).sum()))

vivino_vintages: Foram encontrados 0 registros duplicados pelo ID


In [24]:
print('vivino_vintages: Foram encontrados {} registros duplicados pelo ID do vinho e ano de engarrafamento/safra'\
      .format(vivino_vintages.duplicated(subset=['wine_id', 'year']).sum()))

vivino_vintages: Foram encontrados 0 registros duplicados pelo ID do vinho e ano de engarrafamento/safra


### Tratamento de dados omissos
- Não foram encontrados dados omissos em todo o dataset.

In [25]:
values_missing(vivino_vintages)

,percent_missing,rows_missing


## Unificação dos datasets da Vivino
- Os 207.396 registros de vinhos tintos presentes no dataset Vivino Wines foram enriquecidos com as informações de safras específicas/ano engarramento de VIvino Vintages, o que levou o dataset a 874.676 registros. Importante notar que agora a unicidade do vinho deixa de ser apenas pelo **full_name**, sendo que a partir de agora deve levar em conta o atributo **year**. Isso permitirá na próxima a junção com o dataset da Wine Enthusiast Magazine.

In [26]:
# Merge do dataset Vivino Wines com Vivino Vintages. A coluna id do segundo dataset foi renomeada para vintage_id,
# pois já existe outra coluna id no primeiro dataset. Após o merge, a coluna wine_id usada para o merge foi descartada.
vivino_red = pd.merge(vivino_wines, vivino_vintages.rename({'id': 'vintage_id'}, axis=1), left_on='id', right_on='wine_id')\
            .drop('wine_id', axis=1)


In [27]:
vivino_red.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 874676 entries, 0 to 874675
Data columns (total 18 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       874676 non-null  int64  
 1   full_name                874676 non-null  object 
 2   name                     874676 non-null  object 
 3   wine_region              874676 non-null  object 
 4   winery                   874676 non-null  object 
 5   winery_region            874676 non-null  object 
 6   country                  874676 non-null  object 
 7   ratings_count            874676 non-null  int64  
 8   ratings_average          874676 non-null  float64
 9   alcohol                  874676 non-null  float64
 10  acidity                  874676 non-null  float64
 11  intensity                874676 non-null  float64
 12  sweetness                874676 non-null  float64
 13  tannin                   874676 non-null  float64
 14  year

### Exportação do dataset

In [28]:
# Exporta o dataset para arquivo csv após o tratamento dado até o momento
vivino_red.to_csv('datasets/vivino_red.csv', index=False, header=True)